In [2]:
import sys
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
import os
import matplotlib.pyplot as plt
import numpy as np
import ot
import time
import numba


In [3]:
parent_path='/home/baly/projects/Sliced_opt/code'
sys.path.append(parent_path)
os.chdir(parent_path)
from sopt.library import *
from sopt.lib_shape import *
from sopt.lib_ot import *   
from sopt.sliced_opt import *   


/home/baly/projects/Sliced_opt/code/sopt/sliced_opt.py:105: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 2d, A))
  X_theta=np.dot(theta,X.T)
/home/baly/projects/Sliced_opt/code/sopt/sliced_opt.py:133: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 1d, A), array(float32, 2d, A))
  X_theta=np.dot(theta,X.T)


In [3]:
# our method
#print('here')
@nb.njit(['Tuple((float64[:,:,:],float64[:],float64[:,:]))(float64[:,:],float64[:,:],int64,int64)'])
def sopt_main(S,T,n_iterations,N0):
    n,d=T.shape
    N1=S.shape[0]
    
    # initlize 
    rotation=np.eye(d) #.astype(Dtype)    
    scalar=1.0 #
    beta=vec_mean(T)-vec_mean(scalar*S.dot(rotation)) 
    #paramlist=[]
    projections=random_projections(d,n_iterations,1)
    mass_diff=0
    b=np.log((N1-N0+1)/1)
    Lambda=3*np.sum(beta**2)
    rotation_list=np.zeros((n_iterations,d,d)) #.astype(np.float64)
    scalar_list=np.zeros((n_iterations)) #.astype(np.float64)
    beta_list=np.zeros((n_iterations,d)) #.astype(np.float64)
    T_hat=S.dot(rotation)*scalar+beta
    
    Domain_org=arange(0,N1)
    Delta=Lambda/8
    lower_bound=Lambda/100
    for i in range(n_iterations):
        print('i',i)
        theta=projections[i]
        T_hat_theta=np.dot(theta,T_hat.T)
        T_theta=np.dot(theta,T.T)
        
        T_hat_indice=T_hat_theta.argsort()
        T_indice=T_theta.argsort()
        T_hat_s=T_hat_theta[T_hat_indice]
        T_s=T_theta[T_indice]
        c=cost_matrix(T_hat_s,T_s)
        obj,phi,psi,piRow,piCol=solve_opt(c,Lambda)
        L=piRow.copy()
        L=recover_indice(T_hat_indice,T_indice,L)
        
       # #debug 
       #  if L.max()>=n:
       #      print('error')
       #      return T_hat_theta,T_theta,Lambda
       #      break
        
        #move T_hat
        Domain=Domain_org[L>=0]
        mass=Domain.shape[0]
        if Domain.shape[0]>=1:
            Range=L[L>=0]
            T_hat_take_theta=T_hat_theta[Domain]
            T_take_theta=T_theta[Range]
            T_hat[Domain]+=np.expand_dims(T_take_theta-T_hat_take_theta,1)*theta

        T_hat_take=T_hat[Domain]
        S_take=S[Domain]
        
        rotation,scalar_d=recover_rotation(T_hat_take,S_take)
        scalar=np.sqrt(np.trace(np.cov(T_hat_take.T))/np.trace(np.cov(S_take.T)))
        beta=vec_mean(T_hat_take)-vec_mean(scalar*S_take.dot(rotation))
        T_hat=S.dot(rotation)*scalar+beta
       
        rotation_list[i]=rotation
        scalar_list[i]=scalar
        beta_list[i]=beta
        N=(N1-N0)*1/(1+b*(i/n_iterations))+N0
        mass_diff=mass-N
        if mass_diff>N*0.009:
            Lambda-=Delta 
        if mass_diff<-N*0.003:
            Lambda+=Delta
            Delta=Lambda*1/8
        if Lambda<lower_bound:
            Lambda=lower_bound
    return rotation_list,scalar_list,beta_list    


/tmp/ipykernel_2135918/1457763835.py:11: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 2d, C))
  beta=vec_mean(T)-vec_mean(scalar*S.dot(rotation))
/tmp/ipykernel_2135918/1457763835.py:28: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 2d, F))
  T_hat_theta=np.dot(theta,T_hat.T)
/tmp/ipykernel_2135918/1457763835.py:60: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 2d, A))
  beta=vec_mean(T_hat_take)-vec_mean(scalar*S_take.dot(rotation))
/tmp/ipykernel_2135918/1457763835.py:61: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 2d, A))
  T_hat=S.dot(rotation)*scalar+beta


In [4]:
# our method
#print('here')
@nb.njit(['Tuple((float32[:,:,:],float32[:],float32[:,:]))(float32[:,:],float32[:,:],int64,int64)'])
def sopt_main_32(S,T,n_iterations,N0):
    n,d=T.shape
    N1=S.shape[0]
    
    # initlize 
    rotation=np.eye(d,dtype=np.float32)    
    scalar=np.float32(1) 
    beta=vec_mean_32(T)-vec_mean_32(scalar*S.dot(rotation)) 
    #paramlist=[]
    projections=random_projections_32(d,n_iterations,1)
    mass_diff=0
    b=np.float32(np.log((N1-N0+1)/1))
    Lambda=3*np.sum(beta**2)
    rotation_list=np.zeros((n_iterations,d,d),dtype=np.float32)
    scalar_list=np.zeros((n_iterations),dtype=np.float32)
    beta_list=np.zeros((n_iterations,d),dtype=np.float32)
    T_hat=S.dot(rotation)*scalar+beta
    Domain_org=arange(0,N1)
    Delta=Lambda/8
    lower_bound=Lambda/100
    for i in range(n_iterations):
#        print('i',i)
        theta=projections[i]
        T_hat_theta=np.dot(theta,T_hat.T)
        T_theta=np.dot(theta,T.T)
        
        T_hat_indice=T_hat_theta.argsort()
        T_indice=T_theta.argsort()
        T_hat_s=T_hat_theta[T_hat_indice]
        T_s=T_theta[T_indice]
        c=cost_matrix(T_hat_s,T_s)
        obj,phi,psi,piRow,piCol=solve_opt_32(c,Lambda)
        L=piRow.copy()
        L=recover_indice(T_hat_indice,T_indice,L)
        
#       debug 
#        if L.max()>=n:
#            print('error')
#            return T_hat_theta,T_theta,Lambda
#            break
        
        #move T_hat
        Domain=Domain_org[L>=0]
        mass=Domain.shape[0]
        if Domain.shape[0]>=1:
            Range=L[L>=0]
            T_hat_take_theta=T_hat_theta[Domain]
            T_take_theta=T_theta[Range]
            T_hat[Domain]+=np.expand_dims(T_take_theta-T_hat_take_theta,1)*theta

        T_hat_take=T_hat[Domain]
        S_take=S[Domain]
        
        # compute the optimal rotation, scaling, shift
        rotation,scalar=recover_rotation_32(T_hat_take,S_take)
        #scalar=np.sqrt(np.trace(np.cov(T_hat_take.T))/np.trace(np.cov(S_take.T)))
        beta=vec_mean_32(T_hat_take)-vec_mean_32(scalar*S_take.dot(rotation))
        T_hat=S.dot(rotation)*scalar+beta
       
        rotation_list[i]=rotation
        scalar_list[i]=scalar
        beta_list[i]=beta
        N=(N1-N0)*1/(1+b*(i/n_iterations))+N0
        mass_diff=mass-N
        if mass_diff>N*0.009:
            Lambda-=Delta 
        if mass_diff<-N*0.003:
            Lambda+=Delta
            Delta=Lambda*1/8
        if Lambda<lower_bound:
            Lambda=lower_bound
    return rotation_list,scalar_list,beta_list   

/tmp/ipykernel_2135918/2275447841.py:11: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 2d, A), array(float32, 2d, C))
  beta=vec_mean_32(T)-vec_mean_32(scalar*S.dot(rotation))
/tmp/ipykernel_2135918/2275447841.py:27: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 1d, A), array(float32, 2d, F))
  T_hat_theta=np.dot(theta,T_hat.T)
/tmp/ipykernel_2135918/2275447841.py:60: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 2d, C), array(float32, 2d, A))
  beta=vec_mean_32(T_hat_take)-vec_mean_32(scalar*S_take.dot(rotation))
/tmp/ipykernel_2135918/2275447841.py:61: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 2d, A), array(float32, 2d, A))
  T_hat=S.dot(rotation)*scalar+beta


In [5]:
# # method of spot_boneel method
@nb.njit(['Tuple((float64[:,:,:],float64[:],float64[:,:]))(float64[:,:],float64[:,:],int64,int64)'])
def spot_bonneel(S,T,n_projections=20,n_iterations=200):
    n,d=T.shape
    N1=S.shape[0]
    # initlize 
    rotation=np.eye(d) #,dtype=np.float64)
    scalar=nb.float64(1.0) #
    beta=vec_mean(T)-vec_mean(scalar*S.dot(rotation))
    #paramlist=[]
    
    rotation_list=np.zeros((n_iterations,d,d)) #.astype(np.float64)
    scalar_list=np.zeros((n_iterations)) #.astype(np.float64)
    beta_list=np.zeros((n_iterations,d)) #.astype(np.float64)
    T_hat=S.dot(rotation)*scalar+beta
    
    #Lx_hat_org=arange(0,n)
    
    for i in range(n_iterations):
#        print('i',i)

        projections=random_projections(d,n_projections,1)
        
# #        print('start1')
        T_hat=X_correspondence_pot(T_hat,T,projections)
        rotation,scalar=recover_rotation(T_hat,S)
        beta=vec_mean(T_hat)-vec_mean(scalar*S.dot(rotation))
        T_hat=S.dot(rotation)*scalar+beta

#         #move That         
        rotation_list[i]=rotation         
        scalar_list[i]=scalar
        beta_list[i]=beta

    return rotation_list,scalar_list,beta_list    



In [6]:
@nb.njit(['Tuple((float32[:,:,:],float32[:],float32[:,:]))(float32[:,:],float32[:,:],int64,int64)'])
def spot_bonneel_32(S,T,n_projections=20,n_iterations=200):
    n,d=T.shape
    N1=S.shape[0]
    # initlize 
    rotation=np.eye(d,dtype=np.float32)
    scalar=np.float32(1) 
    beta=vec_mean_32(T)-vec_mean_32(scalar*S.dot(rotation))
    #paramlist=[]
    
    rotation_list=np.zeros((n_iterations,d,d),dtype=np.float32)
    scalar_list=np.zeros((n_iterations),dtype=np.float32)
    beta_list=np.zeros((n_iterations,d),dtype=np.float32)
    T_hat=S.dot(rotation)*scalar+beta
    
    #Lx_hat_org=arange(0,n)
    
    for i in range(n_iterations):
#        print('i',i)
        projections=random_projections_32(d,n_projections,1)
        T_hat=X_correspondence_pot_32(T_hat,T,projections)
        rotation,scalar=recover_rotation_32(T_hat,S)
        beta=vec_mean_32(T_hat)-vec_mean_32(scalar*S.dot(rotation))
        T_hat=S.dot(rotation)*scalar+beta

        #move That         
        rotation_list[i]=rotation         
        scalar_list[i]=scalar
        beta_list[i]=beta

    return rotation_list,scalar_list,beta_list    

In [7]:
@nb.njit(['Tuple((float64[:,:,:],float64[:],float64[:,:]))(float64[:,:],float64[:,:],int64)'])
def icp_du(S,T,n_iterations):
    n,d=T.shape

    # initlize 
    rotation=np.eye(d) #,dtype=np.float64)
    scalar=1.0  #nb.float64(1) #
    beta=vec_mean(T)-vec_mean(scalar*np.dot(S,rotation))

    
    
    rotation_list=np.zeros((n_iterations,d,d)) #.astype(np.float64)
    scalar_list=np.zeros((n_iterations)) #.astype(np.float64)
    beta_list=np.zeros((n_iterations,d)) #.astype(np.float64)
    T_hat=np.dot(S,rotation)*scalar+beta
    
    # #Lx_hat_org=arange(0,n)
    
    for i in range(n_iterations):
#        print('i',i)
        M=cost_matrix_d(T_hat,T)
        argmin_T=closest_y_M(M)
        T_take=T[argmin_T]
        T_hat=T_take
        rotation,scalar_d=recover_rotation_du(T_hat,S)
        scalar=np.mean(scalar_d)
        beta=vec_mean(T_hat)-vec_mean(scalar*S.dot(rotation))
        T_hat=S.dot(rotation)*scalar+beta
        
        #move Xhat         
        rotation_list[i]=rotation
        scalar_list[i]=scalar
        beta_list[i]=beta

    return rotation_list,scalar_list,beta_list  

/tmp/ipykernel_2135918/1457250746.py:8: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 2d, C))
  beta=vec_mean(T)-vec_mean(scalar*np.dot(S,rotation))


In [8]:
@nb.njit(['Tuple((float32[:,:,:],float32[:],float32[:,:]))(float32[:,:],float32[:,:],int64)'])
def icp_du_32(S,T,n_iterations):
    n,d=T.shape

    # initlize 
    rotation=np.eye(d,dtype=np.float32)
    scalar=nb.float32(1) #
    beta=vec_mean_32(T)-vec_mean_32(scalar*np.dot(S,rotation))
    
    
    rotation_list=np.zeros((n_iterations,d,d),dtype=np.float32)
    scalar_list=np.zeros(n_iterations,dtype=np.float32)
    beta_list=np.zeros((n_iterations,d), dtype=np.float32)
    T_hat=np.dot(S,rotation)*scalar+beta
    
    # #Lx_hat_org=arange(0,n)
    
    for i in range(n_iterations):
#        print('i',i)
        M=cost_matrix_d_32(T_hat,T)
#        argmin_T=closest_y_M(M)
#        T_take=T[argmin_T]
#        T_hat=T_take
#        rotation,scalar_d=recover_rotation_du_32(T_hat,S)
#        scalar=np.mean(scalar_d)
#        beta=vec_mean_32(T_hat)-vec_mean_32(scalar*S.dot(rotation))
#        T_hat=S.dot(rotation)*scalar+beta
        
        #move Xhat         
        rotation_list[i]=rotation
        scalar_list[i]=scalar
        beta_list[i]=beta

    return rotation_list,scalar_list,beta_list  

/tmp/ipykernel_2135918/1878762495.py:8: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 2d, A), array(float32, 2d, C))
  beta=vec_mean_32(T)-vec_mean_32(scalar*np.dot(S,rotation))


In [9]:
@nb.njit(['Tuple((float64[:,:,:],float64[:],float64[:,:]))(float64[:,:],float64[:,:],int64)'])
def icp_umeyama(S,T,n_iterations):
    n,d=S.shape

    # initlize 
    rotation=np.eye(d) #,dtype=np.float64)
    scalar=1.0 #nb.float64(1.0) #
    beta=vec_mean(T)-vec_mean(scalar*S.dot(rotation))
    # paramlist=[]
    rotation_list=np.zeros((n_iterations,d,d)) #.astype(np.float64)
    scalar_list=np.zeros((n_iterations)) #.astype(np.float64)
    beta_list=np.zeros((n_iterations,d)) #.astype(np.float64)
    T_hat=S.dot(rotation)*scalar+beta
    

    
    for i in range(n_iterations):
#        print('i',i)
       # print(i)
        M=cost_matrix_d(T_hat,T)
        argmin_T=closest_y_M(M)
        T_take=T[argmin_T]
        T_hat=T_take
        rotation,scalar=recover_rotation(T_hat,S)
        #scalar=np.mean(scalar_d)
        beta=vec_mean(T_hat)-vec_mean(scalar*S.dot(rotation))
        X_hat=S.dot(rotation)*scalar+beta
        
        #move That         
        rotation_list[i]=rotation
        scalar_list[i]=scalar
        beta_list[i]=beta

    return rotation_list,scalar_list,beta_list  

In [10]:
@nb.njit(['Tuple((float32[:,:,:],float32[:],float32[:,:]))(float32[:,:],float32[:,:],int64)'])
def icp_umeyama_32(S,T,n_iterations):
    n,d=S.shape

    # initlize 
    rotation=np.eye(d,dtype=np.float32)
    scalar=nb.float32(1) 
    beta=vec_mean_32(T)-vec_mean_32(scalar*S.dot(rotation))
    # paramlist=[]
    rotation_list=np.zeros((n_iterations,d,d),dtype=np.float32)
    scalar_list=np.zeros((n_iterations),dtype=np.float32)
    beta_list=np.zeros((n_iterations,d),dtype=np.float32)
    T_hat=S.dot(rotation)*scalar+beta
        
    for i in range(n_iterations):
#        print('i',i)
       # print(i)
        M=cost_matrix_d_32(T_hat,T)
        argmin_T=closest_y_M(M)
        T_take=T[argmin_T]
        T_hat=T_take
        rotation,scalar=recover_rotation_32(T_hat,S)
        #scalar=np.mean(scalar_d)
        beta=vec_mean_32(T_hat)-vec_mean_32(scalar*S.dot(rotation))
        X_hat=S.dot(rotation)*scalar+beta
        
        #move That         
        rotation_list[i]=rotation
        scalar_list[i]=scalar
        beta_list[i]=beta

    return rotation_list,scalar_list,beta_list  

In [11]:
item_list=['/stanford_bunny','/dragon','/mumble_sitting','/witchcastle']

#exp_num=item

label_L=['0','1','2','3']
L=['/10k','/9k','/8k','/7k']
time_list={}
(label,per_s) =('1','-5p')

n_point=L[int(label)]    
#data_path=parent_path+'/experiment/shape_registration/data/test2/saved'
#save_path='experiment/shape_registration/result'+exp_num+n_point
#data=torch.load(data_path+item+'.pt')

try:
    time_list=torch.load('experiment/shape_registration/result/time_list.pt')
except:
    time_list={}
    torch.save(time_list,'experiment/shape_registration/result/time_list.pt')
#print('data',item)
#print('experiment/shape_registration/result'+str(item)+'time_list.pt')
#print('time_list',time_list)
#print('data is',item)
#T0=data['T0'].to(torch.float64)
#S0=data['S0'+label].to(torch.float64)
#T1=data['T1'+per_s].to(torch.float64)
#S1=data['S1'+label+per_s].to(torch.float64)


In [12]:
item_list=['/stanford_bunny','/dragon','/mumble_sitting','/witchcastle']

#exp_num=item

label_L=['0','1','2','3']
L=['/10k','/9k','/8k','/7k']
time_list={}
(label,per_s) =('1','-5p')

n_point=L[int(label)]    
#data_path=parent_path+'/experiment/shape_registration/data/test2/saved'
#save_path='experiment/shape_registration/result'+exp_num+n_point
#data=torch.load(data_path+item+'.pt')

try:
    time_list32=torch.load('experiment/shape_registration/result/time_list32.pt')
except:
    time_list32={}    
    torch.save(time_list32,'experiment/shape_registration/result/time_list32.pt')



In [13]:
for key in time_list32['sopt']:
    print(key)
    print(time_list32['sopt'][key])
    
# for key in time_list32['icp-du']:
#     print(key)
#     print(time_list32['icp-du'][key])
    

/stanford_bunny/10k-7p
{'wall_time': 698.769745349884, 'n_iterations': 2000, 'per_time': 0.349384872674942}
/stanford_bunny/9k-5p
{'wall_time': 625.4356114864349, 'n_iterations': 2000, 'per_time': 0.31271780574321745}
/dragon/10k-7p
{'wall_time': 694.2198712825775, 'n_iterations': 2000, 'per_time': 0.3471099356412888}
/dragon/9k-5p
{'wall_time': 611.7947218418121, 'n_iterations': 2000, 'per_time': 0.3058973609209061}
/mumble_sitting/10k-7p
{'wall_time': 713.1314492225647, 'n_iterations': 2000, 'per_time': 0.35656572461128233}
/mumble_sitting/9k-5p
{'wall_time': 636.0941679477692, 'n_iterations': 2000, 'per_time': 0.3180470839738846}
/witchcastle/10k-7p
{'wall_time': 703.1635925769806, 'n_iterations': 2000, 'per_time': 0.3515817962884903}
/witchcastle/9k-5p
{'wall_time': 614.700935125351, 'n_iterations': 2000, 'per_time': 0.3073504675626755}


In [7]:
# result['item']=item
# result['type']=n_point+per_s
# result['wall_time']=wall_time
# result['n_iterations']=n_iterations
# result['per_time']=wall_time/n_iterations

# time_list['sopt']=result
# print(time_list)
# time_list={}



In [15]:
time_list32['sopt']={}
print('hello')
for item in item_list:
    for (label,per_s) in [('0','-7p'),('1','-5p')]:
        n_point=L[int(label)]    
        data_path=parent_path+'/experiment/shape_registration/data/test2/saved'
        save_path='experiment/shape_registration/result'+item+n_point     
        data=torch.load(data_path+item+'.pt')

        time_dict={}

        T0=data['T0'].to(torch.float32)
        S0=data['S0'+label].to(torch.float32)
        T1=data['T1'+per_s].to(torch.float32)
        S1=data['S1'+label+per_s].to(torch.float32)
        print('sopt')

        T=T1.numpy().copy()
        S=S1.numpy().copy()
        N0=S0.shape[0]
        start_time=time.time()
        n_iterations=2000
        sopt_main_32(S,T,n_iterations,N0)
        end_time=time.time()
        wall_time=end_time-start_time

        result={}

        result['wall_time']=wall_time
        result['n_iterations']=n_iterations
        result['per_time']=wall_time/n_iterations

        time_list32['sopt'][item+n_point+per_s]=result
        print('per_s',wall_time/n_iterations)
        print('end')
        torch.save(time_list32,'experiment/shape_registration/result/time_list32.pt')


hello
sopt
per_s 0.349384872674942
end
sopt
per_s 0.31271780574321745
end
sopt
per_s 0.3471099356412888
end
sopt
per_s 0.3058973609209061
end
sopt
per_s 0.35656572461128233
end
sopt
per_s 0.3180470839738846
end
sopt
per_s 0.3515817962884903
end
sopt
per_s 0.3073504675626755
end


In [1]:
print('hello')

hello


In [ ]:
time_list32['spot']={}

for item in item_list:
    for (label,per_s) in [('0','-7p'),('1','-5p')]:
        n_point=L[int(label)]   
#        time_list32=torch.load('experiment/shape_registration/result/time_list32.pt')
 
        data_path=parent_path+'/experiment/shape_registration/data/test2/saved'
        save_path='experiment/shape_registration/result'+item+n_point     
        data=torch.load(data_path+item+'.pt')
        time_dict={}

        T0=data['T0'].to(torch.float32)
        S0=data['S0'+label].to(torch.float32)
        T1=data['T1'+per_s].to(torch.float32)
        S1=data['S1'+label+per_s].to(torch.float32)
        print('spot')

        n_point=L[int(label)]    
        T=T1.numpy().copy()
        S=S1.numpy().copy()
        n_projections=20
        n_iterations=150
        start_time=time.time()
        spot_bonneel_32(S,T,n_projections,n_iterations)
        end_time=time.time()
        wall_time=end_time-start_time
        

        result={}
        result['wall_time']=wall_time
        result['n_iterations']=n_iterations
        result['per_time']=wall_time/n_iterations
        time_list32['spot'][item+n_point+per_s]=result
        print('end')
        torch.save(time_list,'experiment/shape_registration/result/time_list32.pt')



In [21]:
time_list32['icp-du']={}
time_list32['icp-umeyama']={}
print('here1')

here1


In [14]:
# T=T1.numpy().copy().astype(np.float64)
# S=S1.numpy().copy().astype(np.float64)
# N0=S0.shape[0]
# start_time=time.time()
# n_iterations=80
# Result=sopt_main(S,T,n_iterations,N0)
# end_time=time.time()
# wall_time=end_time-start_time

time_list32['icp-du']={}


for item in item_list:
    for (label,per_s) in [('0','-7p'),('1','-5p')]:
        #time_list=torch.load('experiment/shape_registration/result/time_list.pt')
        n_point=L[int(label)]    
        data_path=parent_path+'/experiment/shape_registration/data/test2/saved'
        save_path='experiment/shape_registration/result'+item+n_point     
        data=torch.load(data_path+item+'.pt')
        time_dict={}
        T0=data['T0'].to(torch.float32)
        S0=data['S0'+label].to(torch.float32)
        T1=data['T1'+per_s].to(torch.float32)
        S1=data['S1'+label+per_s].to(torch.float32)
        print('icp-du')
        T=T1.numpy().copy()
        S=S1.numpy().copy()
        n_iterations=200 #400

        start_time=time.time()
        icp_du_32(S,T,n_iterations)
        end_time=time.time()
        wall_time=end_time-start_time
        result={}
        result['wall_time']=wall_time
        result['n_iterations']=n_iterations
        result['per_time']=wall_time/n_iterations
        time_list32['icp-du'][item+n_point+per_s]=result
        print('end')
        torch.save(time_list32,'experiment/shape_registration/result/time_list32.pt')


icp-du
end
icp-du
end
icp-du
end
icp-du
end
icp-du
end
icp-du
end
icp-du
end
icp-du
end


In [15]:
for key in time_list32['icp-du']:
    print(key)
    print(time_list32['icp-du'][key])

/stanford_bunny/10k-7p
{'wall_time': 152.92571306228638, 'n_iterations': 200, 'per_time': 0.7646285653114319}
/stanford_bunny/9k-5p
{'wall_time': 132.25123119354248, 'n_iterations': 200, 'per_time': 0.6612561559677124}
/dragon/10k-7p
{'wall_time': 153.69736742973328, 'n_iterations': 200, 'per_time': 0.7684868371486664}
/dragon/9k-5p
{'wall_time': 131.8185670375824, 'n_iterations': 200, 'per_time': 0.659092835187912}
/mumble_sitting/10k-7p
{'wall_time': 156.33769536018372, 'n_iterations': 200, 'per_time': 0.7816884768009186}
/mumble_sitting/9k-5p
{'wall_time': 131.16096353530884, 'n_iterations': 200, 'per_time': 0.6558048176765442}
/witchcastle/10k-7p
{'wall_time': 152.22128224372864, 'n_iterations': 200, 'per_time': 0.7611064112186432}
/witchcastle/9k-5p
{'wall_time': 131.49492383003235, 'n_iterations': 200, 'per_time': 0.6574746191501617}


In [17]:
print('hello')

hello


In [16]:
time_list32['icp-umeyama']={}

for item in item_list:
    for (label,per_s) in [('0','-7p'),('1','-5p')]:
        print('icp-umeyama')

        #time_list=torch.load('experiment/shape_registration/result/time_list.pt')
        n_point=L[int(label)]    
        data_path=parent_path+'/experiment/shape_registration/data/test2/saved'
        save_path='experiment/shape_registration/result'+item+n_point     
        data=torch.load(data_path+item+'.pt')

        time_dict={}
        print('start2')
        T0=data['T0'].to(torch.float32)
        S0=data['S0'+label].to(torch.float32)
        T1=data['T1'+per_s].to(torch.float32)
        S1=data['S1'+label+per_s].to(torch.float32)
        
        T=T1.numpy().copy()
        S=S1.numpy().copy()
        n_iterations=200 #400

        start_time=time.time()
        icp_umeyama_32(S,T,n_iterations)
        end_time=time.time()
        wall_time=end_time-start_time
        result={}
        result['wall_time']=wall_time
        result['n_iterations']=n_iterations
        result['per_time']=wall_time/n_iterations
        time_list32['icp-umeyama'][item+n_point+per_s]=result
        print('end')
        torch.save(time_list32,'experiment/shape_registration/result/time_list32.pt')
 


icp-umeyama
start2
end
icp-umeyama
start2
end
icp-umeyama
start2
end
icp-umeyama
start2
end
icp-umeyama
start2
end
icp-umeyama
start2
end
icp-umeyama
start2
end
icp-umeyama
start2
end


In [17]:
for key in time_list32['icp-umeyama']:
    print(key)
    print(time_list32['icp-umeyama'][key])

/stanford_bunny/10k-7p
{'wall_time': 187.6535358428955, 'n_iterations': 200, 'per_time': 0.9382676792144775}
/stanford_bunny/9k-5p
{'wall_time': 161.86839079856873, 'n_iterations': 200, 'per_time': 0.8093419539928436}
/dragon/10k-7p
{'wall_time': 187.6578233242035, 'n_iterations': 200, 'per_time': 0.9382891166210174}
/dragon/9k-5p
{'wall_time': 163.0472617149353, 'n_iterations': 200, 'per_time': 0.8152363085746765}
/mumble_sitting/10k-7p
{'wall_time': 187.7801468372345, 'n_iterations': 200, 'per_time': 0.9389007341861725}
/mumble_sitting/9k-5p
{'wall_time': 165.9294307231903, 'n_iterations': 200, 'per_time': 0.8296471536159515}
/witchcastle/10k-7p
{'wall_time': 189.4406704902649, 'n_iterations': 200, 'per_time': 0.9472033524513245}
/witchcastle/9k-5p
{'wall_time': 164.4068624973297, 'n_iterations': 200, 'per_time': 0.8220343124866486}
